# Uproot Tutorial: Quick Hints

## 1. Opening ROOT Files

```python
import uproot
file = uproot.open("file.root")
print(file.keys())        # → ['Events;1']
print(file.classnames())  # → {'Events;1': 'TTree'}
tree = file["Events"]     # TTree object
```

## 2. Trees and Branches

TTree is a table of events.  
Branches are the columns of the table.

```python
branches = tree.arrays()
print(branches.keys())  # ['nMuon', 'Muon_pt', ...]
nMuon = branches["nMuon"]            # jagged? no → value/event
Muon_pt = branches["Muon_pt"]        # jagged → array per event
```

Access a single event:

```python
record0 = branches[0].tolist()
# dict: {'nMuon': 2, 'Muon_pt': [...], …}
```

## 3. Jagged Arrays (Awkward)

- **Rectangular**: all rows same length  
- **Jagged**: variable-length rows (e.g. different muon counts)

```python
import awkward as ak
flat_pts = ak.flatten(branches["Muon_pt"])
```

## 4. Plotting Histograms

```python
import matplotlib.pyplot as plt

plt.hist(flat_pts, bins=50, histtype="step")
plt.xlabel("Muon pT [GeV]")
plt.ylabel("Events")
plt.title("Muon transverse momentum")
plt.show()
```

## 5. Boolean Masks (Selections)

Create a boolean mask for selection (e.g., only muons with pt > 20 GeV):

```python
mask = branches["Muon_pt"] > 20
```

Apply it to filter muon pts:

```python
selected_pts = branches["Muon_pt"][mask]
```

## 6. Event Filtering

Keep only events with ≥1 muon with pT > 20:

```python
has_good_muon = ak.any(branches["Muon_pt"] > 20, axis=1)
filtered_events = branches[has_good_muon]
```

Number of events that passed the cut:

```python
print(len(filtered_events))  # e.g. 13567
```

## 7. Saving Filtered Data

```python
with uproot.recreate("filtered.root") as fout:
    fout["Events"] = filtered_events
```

This writes a new ROOT file with your selected events.

---

✅ **Tips:**
- `ak.num(array)` gives number of entries per event  
- You can use multiple masks:  
  `mask = (pt > 20) & (eta < 2.4)`
- For selections with two muons, try slicing like `Muon_pt[:, 0]`, `[:, 1]`, etc.

---